# 1.Opis

[TUTAJ NALEŻY DODAĆ OPIS]

# 2. Liczba obiektów, klas, zakresy zmienności atrybutów i wartości statystycznych, poziom wypełnienia kolumn, ilość unikalnych danych itp.



# 3. Korelacje między zmiennymi

# 4. Przygotować dane do analizy, inputować brakujące dane lub usunąć rzadko wypełnione kolumny.

# 5. Przeanalizować podobieństwo między danymi przy pomocy poznanych algorytmów grupowania, wraz z analizą ilości grup.

# 6. Dla zadań klasyfkacji należy przetestować wybrane klasyfikatory pod kątem doboru ich parametrów.

# 7. Ocenić czy do poprawnej klasyfikacji należy wykorzystać wszystkie atrybuty, czy wystarczy ich podzbiór, a może należy stworzyć jakieś nowe dane w oparciu o istniejące?

# 8. W przypadku zadań z ekspoloracji danych należy przetestować różne możliwe przecięcia oraz zwizualizować i opisać otrzymane wyniki.

# Załadowanie bibliotek

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Załadowanie danych

In [11]:
data = pd.read_table('Salaries.csv',sep=',')

# Kolumny z nieinteresującymi wartościami
Kolumna Agency zawiera tę samą wartość, co powoduje, że jest ona nieinteresująca:

In [21]:
len(data.Agency.unique())

1

Kolejną kolumną zawierającą jedną wartość jest Notes:

In [24]:
len(data.Notes.unique())

1